<a href="https://colab.research.google.com/github/Uiuran/engineering_exercises/blob/master/Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decorators

In [16]:
# This decorator extends any function to return a matrix with each element being the argments with the function opered
def funcdec(func):
  def wrapper(*args):      
      if len(args)==2:
        if issubclass(type(args[0]),list) and issubclass(type(args[1]),list):
          return [ [func(j,l) for l in args[0]] for j in args[1]]
        else:
          raise Exception("Error: argments musts be lists.")
      else:
          return "Warning: not enough operands"
  return wrapper

@funcdec
def some_op(a,b):
    return a+b
  
@funcdec
def div(a,b):
    return a//b

print(some_op([10],[2]))
print(div([10,20,4,7],[2,3]))

[[12]]
[[0, 0, 0, 0], [0, 0, 0, 0]]


# Decorators with argments ( a.k.a 3 level decorators )



In [31]:
# This level passes arbitrary argments to the decorator
allowed = ['power','p']
def funcdec(*args,**kwargs):
  power=1.
  p = 0.5
  for a in kwargs:
    if a in allowed:
      if a == 'power':
        power=kwargs[a]
      if a == 'p':
        p=kwargs[a]
    else:
      raise Exception("Keyword arg not allowed")
      
  #This level returns the decorator, the argment is the function that will have functionalities added
  def decorator(func):

    # this level adds functionality with inward argments and decorators outer argment
    def wrapper(*inargs, **inkwargs):      
        if len(inargs)==2:
          if issubclass(type(inargs[0]),list) and issubclass(type(inargs[1]),list):
            return [ [(func(l,j)/(len(inargs[0])*len(inargs[1])-p ))**power for l in inargs[0]] for j in inargs[1]]
          else:
            raise Exception("Error: argments musts be lists.")
        else:
          return "Warning: not enough operands"
    return wrapper
  return decorator

@funcdec()
def some_op(a,b):
    return a+b
  
@funcdec(p=2,power=1.4)
def div(a,b):
    return a//b

print(some_op([10],[2]) )
print(div([10,20,4,7],[2,4]))

[[24.0]]
[[0.7747226539790469, 2.044505341160648, 0.21479800499241808, 0.37892914162759955], [0.21479800499241808, 0.7747226539790469, 0.08139322365509782, 0.08139322365509782]]


In [0]:
# Template for Decorators Classes, using functions get set del attr e __iter__ __next__() to access classes file being passed
# Good to compare against python decorators
class FileWithLogging:

  def __init__(self, file):
    self.file = file

  def writelines(self, strings):
    self.file.writelines(strings)
    print(f'wrote {len(strings)} lines')

  def __iter__(self):
    return self.file.__iter__()

  def __next__(self):
    return self.file.__next__()

  def __getattr__(self, item):
    return getattr(self.__dict__['file'], item)

  def __setattr__(self, key, value):
    if key == 'file':
      self.__dict__[key] = value
    else:
      setattr(self.__dict__['file'], key)

  def __delattr__(self, item):
    delattr(self.__dict__['file'], item)


# Class Decorator (Hybrid of the two above)

Suppose you have a Class that inherit from more abstract parent, such an Audio stream. You can extend this class by using hybrid of classic decorator and python functional decorator without having to use inheritance. Here i implement simple statistics to it. 

The decorator receives argments as usual, returning a function that will pass an class to the new class augmenting its capabilities (as in classical decorators).

Instantiating the new class generates an instance of the decorated class with new functions without the heritance.

In [11]:
import numpy as np
allowed = ['power','p']
# Decorator args
def statsdec(*out,**outkwargs):
  
  power=1.
  p = 0.5
  for a in outkwargs:
    if a in allowed:
      if a == 'power':
        power=outkwargs[a]
      if a == 'p':
        p=outkwargs[a]
    else:
      raise Exception("Keyword arg not allowed")
  # Receive the class being decorated
  def decorated(Cls):

    class Decclass(object):
    
      def __init__(self, *args, **kwargs):
        self.processed = Cls(*args, **kwargs)

      def stats(self):
        l=len(self.processed.stream)
        a=(np.mean(self.processed.stream)**power)/(l-p)
        b=(np.std(self.processed.stream)**power)/(l-p)
        return (a,b)        

      def __iter__(self):
        return self.processed.__iter__()
  
      def __next__(self):
        return self.processed.__next__()
  
      def __getattr__(self, item):
        if 'processed' in self.__dict__:
          return getattr(self.__dict__['processed'], item)
        else:      
          return getattr(self, item)
  
      def __setattr__(self, key, value):
        if key not in self.__dict__:
          self.__dict__[key] = value
        else:
          setattr(self.__dict__['processed'][key], value)
  
      def __delattr__(self, item):
        delattr(self.__dict__['processed'], item)        

    return Decclass
  return decorated

class Stream(list):
  def __init__(self, *args, **kwargs):
    if 'streaming' in kwargs:
      self.stream = kwargs['streaming'] 
    else:
      self.stream = [1,2,3,1,2,3,1,2,3,1,2,3]

# modify audio stream to produce statistics about the stream
@statsdec(power = 0.5, p = 3)
class AudioStream(Stream):
  def __init__(self, *args, **kwargs):
    super().__init__(*args,**kwargs)
    print("streaming")
  def play(self):
    print(self.stream)

audiostream = AudioStream(streaming = [2,.4,3.,3.,2.0,3.0,1.0])
audiostream.play()
# Extended capabilities for statistics to have mean and std of the streaming
print(audiostream.stats())

audiostream.processed.stream = [0.2,3.,0.1,0.3,0.5,0.6]
audiostream.play()
print(audiostream.stats())

streaming
[2, 0.4, 3.0, 3.0, 2.0, 3.0, 1.0]
(0.3585685828003181, 0.24576073240107632)
[0.2, 3.0, 0.1, 0.3, 0.5, 0.6]
(0.2950204010522612, 0.3342783807895759)


# Conclusion

Decorators are not simple adding functionalities as classical decorators, but functional decorators does entire new thing, using the decorated function somewhere inside the new thing. 

In adding new structure in Classes Classic Decorators, you can conserve the structure while adding argments, but apparently you cant define arbitrary argments like argments to the decorated function and to the decorator.